In [1]:
import os
import json
from tqdm.notebook import tqdm
import cv2
import matplotlib.pyplot as plt
import uuid
import subprocess
from multiprocessing import Pool
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import shutil
from timeout_decorator import timeout, TimeoutError

In [2]:
inputFolder = "./input"

In [3]:
def trim(json, likelihood=0.98):
    
    save="./faces/{0}.jpg"
    save_up="./faces_up/{0}.jpg"
    
    try:
        if json["likelihood"]<likelihood:
            return 
        img = cv2.imread(json["file"])
        #img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        x = json["face"]["x"]
        y = json["face"]["y"]
        width = json["face"]["width"]
        height = json["face"]["height"]
        img_up = img[y:y+height, x:x+width,:]
        img_trim = img[y-height//2:y+(height*3)//2, x-width//2:x+(width*3)//2,:]
        

        cv2.imwrite(save.format(str(uuid.uuid4())), img_trim)
        cv2.imwrite(save_up.format(str(uuid.uuid4())), img_up)
    except Exception as e:
        pass
        
@timeout(600)
def _create_dataset(filename):
    if (filename[-4::] in (".jpg",".png"))==False :
        return
    uid = str(uuid.uuid4())
    tmpFolder="tmp-" + uid
    os.makedirs(tmpFolder)
    shutil.copy(os.path.join(inputFolder, filename), os.path.join(tmpFolder, filename))
    cmd = "ruby /animeface-2009/animeface-ruby/proc_folder.rb {0} {0}/{1}.json".format(tmpFolder, uid)
    subprocess.run(cmd.split(" "))
    with open("{0}/{1}.json".format(tmpFolder, uid), "r") as fp:
        for line in fp:
            data=json.loads(line)
            trim(data)
    
    shutil.rmtree(tmpFolder)

def create_dataset(filename):
    try:
        _create_dataset(filename)
    except TimeoutError:
        print("timeout:",filename)

In [ ]:
with Pool() as p:
    imap = p.imap(create_dataset, os.listdir(inputFolder))
    list(tqdm(imap, total=len(os.listdir(inputFolder))))